#### 07 - Serving + Demo (Portfolio Style)

This notebook packages the RAG pipeline into a single callable `ask()` function:
1) Embed the question (Azure OpenAI)
2) Retrieve top-k chunks (Option A brute-force or Option B Vector Search)
3) Build a grounded prompt
4) Generate an answer (Azure OpenAI)
5) Log the request/response to `rag_query_logs`
6) (Optional) Evaluate the answer and write to `rag_evaluations`

Output is a JSON-like dict that can later be used for an API or a simple UI.

In [0]:
%run ./00_install_deps_and_restart

In [0]:
%run ./00_constants

In [0]:
%run ./00_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/948.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━ 655.4/948.6 kB 20.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 942.1/948.6 kB 20.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/113.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/73.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/78.8 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/361.3 kB ? eta -:--:--
   ━━━━━━━━━━━━━

In [0]:
%run ./00_init_openai_client

In [0]:
from typing import List, Dict, Any
from datetime import datetime

# Helpers: validate chunk schema + compact sources

def _validate_chunks(chunks: List[Dict[str, Any]]):
    if not isinstance(chunks, list):
        raise TypeError("retrieved_chunks must be a list[dict]")
    if len(chunks) == 0:
        return
    required = {"chunk_id", "doc_id", "title", "url", "chunk_index", "chunk_text", "category", "score"}
    missing = required - set(chunks[0].keys())
    if missing:
        raise ValueError(f"retrieved_chunks items missing keys: {missing}")

def _compact_sources(chunks: List[Dict[str, Any]]):
    # Return a minimal source list for the final API-like payload
    sources = []
    for c in chunks:
        sources.append({
            "title": c.get("title"),
            "url": c.get("url"),
            "category": c.get("category"),
            "chunk_id": c.get("chunk_id"),
            "chunk_index": c.get("chunk_index"),
            "score": c.get("score"),
        })
    return sources

In [0]:
# The unified pipeline function: ask():
# embed → retrieve → prompt → Azure OpenAI → log → (optional) evaluate → return JSON. It 
# also includes a simple notebook “UI” cell and a few SQL queries to show metrics.

def ask(
    question: str,
    *,
    k: int = 6,
    retriever: str = "A",     # "A" brute-force, "B" vector search
    do_eval: bool = True,     # run LLM judge + write to rag_evaluations
    filters: dict = None,     # optional metadata filters for Vector Search
    temperature: float = 0.2
) -> dict:
    """
    End-to-end RAG call:
      - Embedding (Azure OpenAI)
      - Retrieval (A or B)
      - Prompt assembly
      - Chat completion (Azure OpenAI)
      - Logging (rag_query_logs)
      - Optional evaluation (rag_evaluations)

    Returns:
      {
        query_id: str,
        question: str,
        answer: str,
        sources: [...],
        eval: {...} | None
      }
    """
    if not isinstance(question, str) or not question.strip():
        raise ValueError("question must be a non-empty string")

    # 1) Embed
    q_emb = embed_texts(question)

    # 2) Retrieve (returns list[dict])
    if retriever.upper() == "B":
        chunks = retrieve_top_k(q_emb, option="B", k=k, filters=filters)
    else:
        chunks = retrieve_top_k(q_emb, option="A", k=k)

    _validate_chunks(chunks)

    # 3) Prompt
    prompt = build_prompt(question, chunks)

    # 4) Call LLM
    resp = aoai.chat.completions.create(
        model=CHAT_DEPLOYMENT,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    answer = resp.choices[0].message.content

    # 5) Log
    rag_event = {
        "question": question,
        "top_k": k,
        "retrieved_chunks": chunks,
        "prompt": prompt,
        "answer": answer,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "chat_deployment": CHAT_DEPLOYMENT
    }
    query_id = log_rag_event(rag_event)

    # 6) Optional evaluation
    eval_result = None
    if do_eval:
        # judge_rag expects chunk_text to exist; ensure it's present
        eval_scores = judge_rag(question, answer, chunks)
        eval_id = write_evaluation(
            query_id=query_id,
            question=question,
            answer=answer,
            scores=eval_scores,
            evaluator="llm_judge_v1"
        )
        eval_result = {
            "evaluation_id": eval_id,
            **eval_scores
        }

    return {
        "query_id": query_id,
        "question": question,
        "answer": answer,
        "sources": _compact_sources(chunks),
        "eval": eval_result
    }

In [0]:
# Notebook “UI” demo cell

question = "How do I resize an Azure virtual machine?"
result = ask(question, k=6, retriever="A", do_eval=True)

print("Query ID:", result["query_id"])
print("\nAnswer:\n", result["answer"])

print("\nTop sources:")
for s in result["sources"][:3]:
    print("-", s["url"])

Trace(request_id=tr-12937a50144345fb9de7aa1f46b2f8f7)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-5573401567029324>, line 4
      1 # Notebook “UI” demo cell
      3 question = "How do I resize an Azure virtual machine?"
----> 4 result = ask(question, k=6, retriever="A", do_eval=True)
      6 print("Query ID:", result["query_id"])
      7 print("\nAnswer:\n", result["answer"])

File <command-5573401567029257>, line 47, in ask(question, k, retriever, do_eval, filters, temperature)
     44 _validate_chunks(chunks)
     46 # 3) Prompt
---> 47 prompt = build_prompt(question, chunks)
     49 # 4) Call LLM
     50 resp = aoai.chat.completions.create(
     51     model=CHAT_DEPLOYMENT,
     52     messages=[{"role": "user", "content": prompt}],
     53     temperature=temperature
     54 )

File <command-5573401567029282>, line 4, in build_prompt(question, contexts)
      3 def build_prompt(question, contexts):
----> 4     jo

In [0]:
# Try a few:
qs = [
    "What is an availability set used for?",
    "How do I change VM size in Azure?",
    "How do I create a VM from an image?",
]
for q in qs:
    out = ask(q, k=6, retriever="A", do_eval=True)
    print("\n======================")
    print("Q:", q)
    print("A:", out["answer"][:400], "...")
    print("Faithfulness:", out["eval"]["faithfulness"] if out["eval"] else None)

In [0]:
# Latest logs

spark.sql(f"""
SELECT
  created_at,
  query_id,
  question,
  top_k,
  retrieved_chunks[0].url AS top_source
FROM {RAG_LOG_TABLE}
ORDER BY created_at DESC
LIMIT 20
""").display()

In [0]:
# Latest evaluations

spark.sql(f"""
SELECT
  created_at,
  query_id,
  retrieval_relevance,
  answer_relevance,
  faithfulness,
  notes
FROM {RAG_EVAL_TABLE}
ORDER BY created_at DESC
LIMIT 20
""").display()